このNotebookは、モデルを学習させるために作られたものである。

# 1. Import

In [6]:
import os
import random
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
from collections import defaultdict
from typing import List, Dict, Optional, Tuple
from IPython.display import display
import datetime
import time
from tqdm.notebook import tqdm

# Data handling
import numpy as np
import polars as pl
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from skmultilearn.model_selection import iterative_train_test_split

# Medical imaging
import pydicom
import cv2

# Machine Lerning 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import autocast
import torchvision
import timm

# Transformations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import PIL.Image as Image

# Experiment Management
import wandb

# Competition API
# import kaggle_evaluation.rsna_inference_server


# 2. Configuration

In [7]:
# datetime for unique checkpoint filenames
date_time = datetime.datetime.now()
date_time = date_time.strftime('%Y-%m-%d_%H-%M-%S')


In [8]:
# Run Configuration
RUN_NAME = "swin-s-meta-pretrained"
SAVE_DIR = "../results"
TEST_RUN = True
SEED = 42
DEVICE = "cuda"

# Input Data Configuration
IMAGE_SIZE = 256
NUM_SLICES = 32
BATCH_SIZE = 5
LABEL_NAMES = [
    # 13 classes
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    # 'Aneurysm Present',
]
NUM_LABELS = len(LABEL_NAMES)

# Training Configuration
NUM_EPOCHS = 20
PATIENCE = 5


In [9]:
RUN_NAME = RUN_NAME + f'-{IMAGE_SIZE}-{NUM_SLICES}'
SAVE_DIR = SAVE_DIR + '/' + RUN_NAME + f'-{date_time}'

# Weights & Biases Configuration
if TEST_RUN:
    USE_WANDB = False
    WANDB_INIT = {}
    ARTIFACT = {}
else:
    USE_WANDB = True
    WANDB_INIT = {
        'project': 'RSNA-IAD',
        'group': 'Image Classification',
        'job_type': 'training_model',
        'save_code': True,
    }
    ARTIFACT = {
        'name': RUN_NAME,
        'type': 'model, optimizer, scheduler',
    }

In [10]:
class Configuration:
    
    # Run
    run_name = RUN_NAME
    save_dir = SAVE_DIR
    test_run = TEST_RUN
    seed = SEED
    device = DEVICE
    
    # Input Data
    image_size = IMAGE_SIZE
    num_slices = NUM_SLICES
    batch_size = BATCH_SIZE
    label_names = LABEL_NAMES
    num_labels = NUM_LABELS
    
    # Training
    num_epochs = NUM_EPOCHS
    patience = PATIENCE
    
    # Weights & Biases
    use_wandb = USE_WANDB
    wandb_init = WANDB_INIT
    artifact = ARTIFACT

CFG = Configuration


In [11]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device == torch.device('cuda'):
    CFG.device = 'cuda'
    print(f"Using device: {CFG.device}")
else:
    print("CUDA is not available. Using CPU instead.")


Using device: cuda


In [12]:
def set_random_seed(seed=CFG.seed, deterministic=False):
    """
    Set random seed.
    
    Args:
        seed (int): Seed to be used.
        deterministic (bool): Whether to set the deterministic option for
            CUDNN backend, i.e., set `torch.backends.cudnn.deterministic`
            to True and `torch.backends.cudnn.benchmark` to False.
            Default: False.
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
    if deterministic:
        torch.backends.cudnn.benchmark = True


In [13]:
set_random_seed(seed=CFG.seed, deterministic=True)


# 3. Weights & Biases

In [14]:
if CFG.use_wandb:
    os.environ['WANDB_NOTEBOOK_NAME'] = CFG.run_name
    wandb.login()
    run = wandb.init(**CFG.wandb_init)
    artifact = wandb.Artifact(**CFG.artifact)
else:
    run = None
    artifact = None


In [15]:
def alert_by_wandb(title='', text=''):
    wandb.alert(title, text)


# 4. Model

In [16]:
class SwinWithMetaModel(nn.Module):
    def __init__(self, model_name, pretrained=False,
                 num_classes=CFG.num_labels, drop_rate=0.3,
                 drop_path_rate=0.2):
        super().__init__()
        self.model_name = model_name
        
        if model_name == 'swin_s':
            self.backbone = timm.create_model(
                'swin_small_patch4_window7_224',
                pretrained=pretrained,
                img_size=CFG.image_size,
                drop_rate=drop_rate,
                drop_path_rate=drop_path_rate,
                global_poopling='',
                num_classes=0)
            
            # input layer modification: 3 channels -> CFG.num_slices channels
            self.backbone.patch_embed.proj = nn.Conv2d(
                in_channels=CFG.num_slices,
                out_channels=96,
                kernel_size=4,
                stride=4,
            )
        else:
            raise ValueError(f"Model {model_name} is not supported.")
        
        self.meta_features = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(16, 32),
            nn.ReLU()
        )
        
        # According to "LB #1"
        self.classifier = nn.Sequential(
            nn.Linear(768 + 32, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, images, meta): 
        image_features = self.backbone(images)
        meta_fieatures = self.meta_features(meta)
        x = torch.cat([image_features, meta_fieatures], dim=1)
        x = self.classifier(x)
        x = torch.nn.Sigmoid()(x)
        return x

model = SwinWithMetaModel(model_name='swin_s', pretrained=True)

-> timm.createmodel(num_classes=0)とすると、最後のnn.Linear()がnn.Identity()になる。

In [17]:
model.to(CFG.device)

is_in_cuda_list = []

for name, parameter in model.named_parameters():
    # determination of cuda and its storage
    is_in_cuda_list.append(parameter.is_cuda)
    
if all(is_in_cuda_list):
    print('All parameters is in cuda')
        
else:
    print('One of the parameters is not in the cuda.')


All parameters is in cuda


In [18]:
# Optimizer
optimizer = torch.optim.AdamW(model.parameters())

# Loss Function
criterion = nn.BCEWithLogitsLoss()

# Schedulers
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=CFG.num_epochs,
    eta_min=1e-6
)


# 5. Dataset

In [19]:
# SeriesInstanceUID list
series_list = os.listdir(f'../series_npy/{CFG.image_size}')

# .npy path DataFrame
image_path_df = pd.read_csv(f'../npy_path/image_{CFG.image_size}_path_df.csv')

# Meta DataFrame
meta_df = pd.read_csv('../meta_data/meta.csv')

# Label DataFrame
label_df = pd.read_csv(f'../train.csv')
label_df = label_df[['SeriesInstanceUID'] + CFG.label_names]


In [20]:
meta_df.loc[meta_df['SeriesInstanceUID'] == '1.2.826.0.1.3680043.8.498.98697915765488213704603518081182644986']


,SeriesInstanceUID,modalisy,age,sex
4296,1.2.826.0.1.3680043.8.498.98697915765488213704...,MR,60,1


In [21]:
# for training
train_transform = A.Compose(
    [
        # # Elastic Transform <- あとで試したい
        # A.ElasticTransform( p=0.5),
        
        # Rotation
        A.Rotate(limit=(-3, 3), p=0.5, border_mode=cv2.BORDER_WRAP,  # cv2.BORDER_WRAP,
                 seed=CFG.seed
        ),
        
        # Normalization
        A.Normalize(normalization='min_max'),
        
        # ToTensor
        ToTensorV2(),
    ]
)

# for inference
inference_transform = A.Compose(
    [
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ]
)
    
# for TTA
tta_transform = A.Compose(
    [
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            
        # Horizontal flip
        A.HorizontalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        
        # Vertical flip
        A.VerticalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        
        # 90 degree rotation
        A.RandomRotate90(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        
        # ↓ Original
        # Sharpen
        A.Sharpen(alpha=(0, 1.0), p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        
        ToTensorV2(),
    ]
)


In [22]:
class BaseDataset(torch.utils.data.Dataset):
    '''
    Datasetの__getitem__()は、num_slicesの枚数分だけ画像を出力する。
    
    Arguments:
    - series_list: 画像のSeriesInstanceUIDのリスト
    - image_path_df: 画像のパスを含むDataFrame
    - meta_df: 患者のメタデータが入ったDataFrame
    - label_df: ラベルが入ったDataFrame
    - num_slices: 1つのシリーズから抽出するスライス数
    - transforms: 画像変換のためのAlbumentationsのComposeオブジェクト
    '''
    def __init__(self,
                 series_list: list,
                 image_path_df=image_path_df,
                 meta_df=meta_df,
                 label_df=label_df,
                 transforms=None
        ):
        self.series_list = series_list
        self.image_path_df = image_path_df
        self.meta_df = meta_df
        self.label_df = label_df
        self.transforms = transforms
        self.num_slices = CFG.num_slices

    def __len__(self):
        return len(self.series_list)

    def __getitem__(self, index):
        # Index to SeriesInstanceUID
        series_id = self.series_list[index]
        
        # SeriesInstanceUID to Image Path
        image_path_df = self.image_path_df[
            self.image_path_df['series_id'] == series_id
        ].reset_index(drop=True)
        
        # Load Images
        indices = np.linspace(0,
                              len(image_path_df) - 1,
                              self.num_slices).astype(np.int32)
        # Stack images to (H, W, CFG.num_slices)
        images = []
        for i in indices:
            image_path = image_path_df.loc[i, 'npy_path']
            image = np.load(image_path).astype(np.uint8)
            images.append(image)
        images = np.stack(images, axis=-1)
        
        # Transform
        if self.transforms:
            # ToTensorV2はnumpy.ndarrayをtorch.Tensorに変換する
            augmented = self.transforms(image=images)
            images = augmented['image']
        else:
            images = torch.tensor(images, dtype=torch.float32)
            images = torch.permute(images, (2, 0, 1))
            # Min-Max Normalization
            if torch.max(images) > 1.0:
                max_value = torch.max(images)
                min_value = torch.min(images)
                images = (images - min_value) / (max_value - min_value)
                
        # Meta data
        meta = self.meta_df.loc[
            self.meta_df['SeriesInstanceUID'] == series_id, ['age', 'sex']
        ]
        age = min(meta['age'].values[0], 100)
        age = age / 100
        sex = meta['sex'].values[0]
        meta = torch.tensor([age, sex], dtype=torch.float32)

        # Labels
        labels = self.label_df.loc[
            self.label_df['SeriesInstanceUID']==series_id, \
                CFG.label_names].values
        labels = torch.tensor(labels, dtype=torch.float32)
        labels = torch.squeeze(labels, dim=0)
        
        return (images, meta, labels)


# 6. DataLoader

In [23]:
def build_dataloaders():

    series = label_df[["SeriesInstanceUID"]].values
    labels = label_df[CFG.label_names].values

    if CFG.test_run:
        # As the absolute number of data points cannot be specified,
        # split is executed in two stages.
        train_series, train_labels, val_series, _ = iterative_train_test_split(
            series, labels, test_size=(1/len(series)) \
                * 2
        )
        _, _, train_series, train_labels = iterative_train_test_split(
            train_series, train_labels, test_size=(1/len(series)) \
                * 2
        )
        
    else:
        train_series, _, val_series, _ = iterative_train_test_split(
            series, labels, test_size=0.2
        )

    # 2 dimensions -> 1 dimension
    train_series, val_series = train_series.flatten(), val_series.flatten()
    print(f"Train size: {len(train_series)}, Val size: {len(val_series)}")

    # Datasets
    train_dataset = BaseDataset(
        series_list=train_series,
        transforms=train_transform
    )
    val_dataset = BaseDataset(
        series_list=val_series,
        transforms=train_transform # or tta_transform
    )
    
    # Dataloaders
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=0
    )
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=0
    )

    return  train_dataloader, val_dataloader


In [24]:
train_dataloader, val_dataloader = build_dataloaders()


Train size: 2, Val size: 2


In [25]:
# _, ax = plt.subplots(1, 2, figsize=(12, 6))

# # 元の画像とどのくらい違いがあるかを確認

# # 元の画像(.npy)
# src = np.load(f'../series_npy/{CFG.image_size}/1.2.826.0.1.3680043.8.498.10034081836061566510187499603024895557/00012.npy')
# print(np.unique(src))
# ax[0].imshow(src)

# # Datasetから取り出した画像
# images, _ = train_dataset[0]
# image = images[8].numpy()  # shape: [H, W]

# # 0-1のfloatなら0-255に変換
# if image.max() <= 1.0:
#     image = (image * 255).astype(np.uint8)
# else:
#     image = image.astype(np.uint8)

# ax[1].imshow(image)


In [26]:
# pil_image = Image.fromarray(image)
# display(pil_image)


# 7. Functions

In [27]:
# count execution time for one epoch
def count_time(start:float) -> float:
    
    elapsed_time = time.time() - start
    elapsed_time /= 60
    
    return elapsed_time


In [28]:
# to save model, optimizer, scheduler
def save_checkpoint(model, optimizer, scheduler, save_dir=CFG.save_dir):
    
    model.to('cpu')
    
    model_state_dict =  model.state_dict()
    optimizer_state_dict = optimizer.state_dict()
    scheduler_state_dict = scheduler.state_dict()
    
    model_path = save_dir + f'/model.pth'
    optimizer_path = save_dir + f'/optimizer.pth'
    scheduler_path = save_dir + f'/scheduler.pth'
        
    torch.save(model_state_dict, model_path)
    torch.save(optimizer_state_dict, optimizer_path)
    torch.save(scheduler_state_dict, scheduler_path)
    
    model.to(device)
    
    print(f"Model saved.")

# to load model, optimizer, scheduler
def load_checkpoint(model, optimizer, scheduler, save_dir=''):
    
    model.to('cpu')
    
    model.load_state_dict(save_dir + '/model.pth')
    optimizer.load_state_dict(save_dir + '/optimizer.pth')
    scheduler.load_state_dict(save_dir + '/scheduler.pth')
    
    model.to(device)
    
    return model, optimizer, scheduler


In [29]:
# to log losses to W&B
def log_by_wandb(time, losses):
    epoch_data = {
        'time': time,
        'loss': losses,
    }
    wandb.log(epoch_data)


In [30]:
# to log checkpoint
def log_artifact(run=run, artifact=artifact, checkpoint_path=""):
    artifact.add_file(checkpoint_path)
    run.log_artifact(artifact)
    print('Artifact was logged to W&B')


# 8. Training

In [31]:
def train_one_epoch(epoch: int) -> Tuple[float, float]:
    
    print(f'----- Epoch {epoch + 1} -----')
    
    # Training
    train_losses = []
    model.train()
    
    for images, meta, labels in tqdm(train_dataloader):    
        images = images.to(CFG.device)
        meta = meta.to(CFG.device)
        labels = labels.to(CFG.device)
        optimizer.zero_grad()
        with autocast(device_type=CFG.device):
            outputs = model(images, meta)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
    
    mean_train_loss = np.mean(train_losses)
    print(f'Last output:', outputs[0])
    print(f'Mean Train Loss: {mean_train_loss:.4f}')
    
    # Validation
    val_losses = []
    model.eval()
    
    with torch.no_grad():
        for images, meta, labels in tqdm(val_dataloader):
            images = images.to(CFG.device)
            meta = meta.to(CFG.device)
            labels = labels.to(CFG.device)
            with autocast(device_type=CFG.device):
                outputs = model(images, meta)
                loss = criterion(outputs, labels)
                val_losses.append(loss.item())
        
    mean_val_loss = np.mean(val_losses)
    print(f'Mean Validation Loss: {mean_val_loss:.4f}')
        
    scheduler.step()
        
    return mean_train_loss, mean_val_loss


In [32]:
def main():
    
    os.makedirs(CFG.save_dir, exist_ok=True)
    best_val_loss = np.inf
    
    for epoch in range(CFG.num_epochs):
        
        # Train & Validation
        start_time = time.time()
        train_loss, val_loss = train_one_epoch(epoch)
        elapsed_time = count_time(start_time)
        print(f'Elapsed time: {elapsed_time}')
        
        # Log to W&B
        if CFG.use_wandb:
            losses = {
                'train_loss': train_loss,
                'val_loss': val_loss
            }
            log_by_wandb(elapsed_time, losses)
        
        # Save best checkpoint
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_checkpoint_path = os.path.join(
                CFG.save_dir,
                f'best_checkpoint_{date_time}.pth'
            )
            best_checkpoint_path = best_checkpoint_path.replace('\\', '/')
            save_checkpoint(model, optimizer, scheduler)
            print(f'Best checkpoint saved at {best_checkpoint_path}')
            
        else:
            # Save checkpoint
            save_checkpoint(model, optimizer, scheduler)
            print(f'Checkpoint saved at {CFG.save_dir}')
    
    # Log artifact to W&B
    if CFG.use_wandb:
        log_artifact(run, artifact, best_checkpoint_path)


In [33]:
main()


----- Epoch 1 -----


  0%|          | 0/1 [00:00<?, ?it/s]

Last output: tensor([0.5947, 0.5010, 0.6284, 0.4407, 0.4519, 0.6421, 0.5044, 0.6499, 0.4644,
        0.4197, 0.4050, 0.3823, 0.4358], device='cuda:0', dtype=torch.float16,
       grad_fn=<SelectBackward0>)
Mean Train Loss: 0.9978


  0%|          | 0/1 [00:00<?, ?it/s]

Mean Validation Loss: 0.9731
Elapsed time: 0.060632773240407306
Model saved.
Best checkpoint saved at ../results/swin-s-meta-pretrained-256-32-2025-09-30_01-43-33/best_checkpoint_2025-09-30_01-43-33.pth
----- Epoch 2 -----


...

Mean Validation Loss: 0.9250
Elapsed time: 0.01095363696416219
Model saved.
Best checkpoint saved at ../results/swin-s-meta-pretrained-256-32-2025-09-30_01-43-33/best_checkpoint_2025-09-30_01-43-33.pth
----- Epoch 20 -----


  0%|          | 0/1 [00:00<?, ?it/s]

Last output: tensor([0.3484, 0.4189, 0.2363, 0.2754, 0.3552, 0.3967, 0.2979, 0.2310, 0.3516,
        0.2559, 0.1782, 0.2981, 0.1985], device='cuda:0', dtype=torch.float16,
       grad_fn=<SelectBackward0>)
Mean Train Loss: 0.8457


  0%|          | 0/1 [00:00<?, ?it/s]

Mean Validation Loss: 0.9245
Elapsed time: 0.012459063529968261
Model saved.
Best checkpoint saved at ../results/swin-s-meta-pretrained-256-32-2025-09-30_01-43-33/best_checkpoint_2025-09-30_01-43-33.pth


In [34]:
if CFG.use_wandb:
    run.finish()
